# Случайный лес

В этом практическом задании вы решите задачу регрессии на реальных данных при помощи линейной модели и случайного леса. 

## Цели практической работы

* Научиться применять случайный лес в задаче регрессии.
* Научиться сравнивать качество случайного леса с качеством линейной модели.
* Научиться настраивать гиперпараметры леса.

## Что входит в работу

* Выполнить предобработку данных.
* Обучить линейную регрессию на данных задачи.
* Обучить случайный лес на данных задачи.
* Подобрать гиперпараметры для леса.
* Визуализировать важность признаков у леса.

## Что оценивается

*  Все ячейки заполнены; при запуске ячеек  Python не выдаёт информацию об ошибках.
*  Качество итоговой модели $R^2$ превышает 0,95.


## Что нужно сделать

Постройте модели, предсказывающие стоимость автомобилей по характеристикам.

Признаков у машин много, но в этой работе обойдёмся только числовыми признаками:
*  year — год производства автомобиля;
*  km_driven — пробег;
*  seats — количество посадочных мест;
*  mileage — другая характеристика пробега;
*  engine — мощность двигателя;
*  max_power — мощность автомобиля.

Целевая переменная — selling_price.

In [190]:
import pandas as pd
import numpy as np

train = pd.read_csv("cars_train.csv")
test = pd.read_csv("cars_test.csv")

Создайте объекты Xtrain, ytrain, Xtest, ytest. Запишите в них матрицы «объект — признак» и векторы целевой переменной для тренировочного и тестового датасетов.

In [191]:
# Ваш код здесь
Xtrain, ytrain, Xtest, ytest = train.drop('selling_price', axis = 1), train.selling_price, test.drop('selling_price', axis = 1), test.selling_price
Xtrain.head(7)

,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014.0,145500.0,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014.0,120000.0,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Hyundai i20 Sportz Diesel,2010.0,127000.0,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
3,"Maruti Swift VXI BSIII,2007,130000,120000,Petr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hyundai Xcent 1.2 VTVT E Plus,2017.0,45000.0,Petrol,Individual,Manual,First Owner,20.14 kmpl,1197 CC,81.86 bhp,113.75nm@ 4000rpm,5.0
5,"Maruti Wagon R LXI DUO BSIII,2007,96000,175000...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Maruti 800 DX BSII,2001.0,5000.0,Petrol,Individual,Manual,Second Owner,16.1 kmpl,796 CC,37 bhp,59Nm@ 2500rpm,4.0


В тренировочных данных есть автомобили, для которых неизвестна стоимость. Удалите эти автомобили из трейна.

In [192]:
# Ваш код здесь
drop = ytrain[ytrain.isna()].index
Xtrain = Xtrain.drop(drop)
ytrain = ytrain.drop(drop)

Перед обучением моделей обработайте данные.

Обратите внимание, что столбцы mileage, engine и max_power по смыслу числовые. Чтобы превратить их в числовые столбцы, отбросьте единицы измерения и оставьте только числа. В столбцах есть пропущенные значения, их при обработке трогать не нужно.

In [193]:
# Ваш код здесь
Xtrain['mileage'] = Xtrain['mileage'].str.replace(r'[^\d\.]', '', regex=True)
Xtrain['engine'] = Xtrain['engine'].str.replace(r'[^\d\.]', '', regex=True)
Xtrain['max_power'] = Xtrain['max_power'].str.replace(r'[^\d\.]', '', regex=True)

Xtest['mileage'] = Xtest['mileage'].str.replace(r'[^\d\.]', '', regex=True)
Xtest['engine'] = Xtest['engine'].str.replace(r'[^\d\.]', '', regex=True)
Xtest['max_power'] = Xtest['max_power'].str.replace(r'[^\d\.]', '', regex=True)

Оставьте в данных только шесть числовых столбцов:

year, km_driven, seats, engine, mileage, max_power

In [194]:
Xtrain.head()

,name,year,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014.0,145500.0,Diesel,Individual,Manual,First Owner,23.4,1248,74,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014.0,120000.0,Diesel,Individual,Manual,Second Owner,21.14,1498,103.52,250Nm@ 1500-2500rpm,5.0
2,Hyundai i20 Sportz Diesel,2010.0,127000.0,Diesel,Individual,Manual,First Owner,23.0,1396,90,22.4 kgm at 1750-2750rpm,5.0
4,Hyundai Xcent 1.2 VTVT E Plus,2017.0,45000.0,Petrol,Individual,Manual,First Owner,20.14,1197,81.86,113.75nm@ 4000rpm,5.0
6,Maruti 800 DX BSII,2001.0,5000.0,Petrol,Individual,Manual,Second Owner,16.1,796,37,59Nm@ 2500rpm,4.0


In [195]:
# Ваш код здесь
Xtrain = Xtrain.drop(['torque','name','fuel','seller_type','transmission','owner'], axis = 1)
Xtest = Xtest.drop(['torque','name','fuel','seller_type','transmission','owner'], axis = 1)

Теперь заполните пропуски следующим образом:

*    вычислите средние значения по столбцам тренировочной выборки;

*    добавьте в пропуски в тренировочных и тестовых данных вычисленные средние.

In [196]:
Xtrain = Xtrain.astype('Float32').astype('Int32')
Xtest = Xtest.astype('Float32').astype('Int32')

In [197]:
# Ваш код здесь
Xtrain.fillna(Xtrain.mean().astype(int), inplace=True)
Xtest.fillna(Xtest.mean().astype(int), inplace=True)
ytrain.fillna(ytrain.mean().astype(int), inplace=True)

In [198]:
Xtrain.isna().sum()

year         0
km_driven    0
mileage      0
engine       0
max_power    0
seats        0
dtype: int64

In [199]:
Xtest.isna().sum()

year         0
km_driven    0
mileage      0
engine       0
max_power    0
seats        0
dtype: int64

Теперь на обработанных тренировочных данных обучите:

*  линейную регрессию,
*  случайный лес с параметрами по умолчанию.

На обработанных тестовых данных сделайте предсказание и вычислите метрику $R^2$. 

In [217]:
# Ваш код здесь
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

lr_model = LinearRegression()
lr_model.fit(Xtrain, ytrain)
lr_pred = lr_model.predict(Xtest)

rf_model = RandomForestRegressor()
rf_model.fit(Xtrain, ytrain)
rf_pred = rf_model.predict(Xtest)

print (f"Метрика r2_score линейной регрессии: {r2_score(ytest, lr_pred)}, \n Метрика r2_score случайного леса: {r2_score(ytest, rf_pred)}")

Метрика r2_score линейной регрессии: 0.6495107307282665, 
 Метрика r2_score случайного леса: 0.9697447610231974


Какая модель получилась лучше? -- Случайный лес

In [234]:
# Ваш ответ здесь
# Модель случайного леса получилась лучше

Улучшите качество предсказания случайного леса путём подбора гиперпараметров:

*   n_estimators,
*   max_depth,
*   max_features,
*   min_samples_leaf,
*   min_samples_split.

Для подбора гиперпараметров используйте GridSearchCV. Обучайте GridSearchCV по тренировочным данным с разбивкой на три фолда и метрикой $R^2$.

In [228]:
# Ваш код здесь
from sklearn.model_selection import GridSearchCV

params = {'n_estimators': np.arange(100, 500, 100),
         'max_depth': np.arange(5, 45, 10),
         'max_features': ['sqrt', 'log2', None],
         'min_samples_leaf': np.arange(1, 21, 4),
         'min_samples_split': np.arange(50, 300, 50)}

gs = GridSearchCV(RandomForestRegressor(), params, cv=5, scoring='r2')

gs.fit(Xtrain, ytrain)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Георгий\AppData\Local\Temp\ipykernel_106116\3646850867.py", line 11, in <module>
    gs.fit(Xtrain, ytrain)
  File "C:\anaconda3\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py", line 1018, in fit
    self._run_search(evaluate_candidates)
  File "C:\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py", line 1572, in _run_search
    evaluate_candidates(ParameterGrid(self.param_grid))
  File "C:\anaconda3\Lib\site-packages\sklearn\model_selection\_search.py", line 964, in evaluate_candidates
    out = parallel(
          ^^^^^^^^^
  File "C:\anaconda3\Lib\site-packages\sklearn\utils\parallel.py", 

In [229]:
rf = RandomForestRegressor(random_state=42)

param_grid = {'n_estimators': np.arange(100, 500, 100),
         'max_depth': np.arange(5, 45, 10),
         'max_features': ['sqrt', 'log2', None],
         'min_samples_leaf': np.arange(1, 21, 4),
         'min_samples_split': np.arange(50, 300, 50)}

grid_search_rf = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    scoring='r2',
    verbose=1,
    n_jobs=-1)

grid_search_rf.fit(Xtrain, ytrain)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


C:\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=RandomForestRegressor(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([ 5, 15, 25, 35]),
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': array([ 1,  5,  9, 13, 17]),
                         'min_samples_split': array([ 50, 100, 150, 200, 250]),
                         'n_estimators': array([100, 200, 300, 400])},
             scoring='r2', verbose=1)

In [230]:
best_params = grid_search_rf.best_params_
best_params

{'max_depth': 25,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 50,
 'n_estimators': 300}

In [232]:
model = grid_search_rf.best_estimator_

Теперь обучите на тренировочных данных случайный лес с найденными гиперпараметрами. Сделайте предсказание на тестовых данных и оцените его качество ($R^2$).

In [236]:
# Ваш код здесь
model.fit(Xtrain, ytrain)

pred = model.predict(Xtest)

r2_score(ytest, pred)

0.9414615034280138

Модель готова. Осталось её проинтерпретировать.

При помощи метода model.feature_importances_ визуализируйте гистограмму важности признаков у случайного леса с настроенными гиперпараметрами.

In [243]:
# Ваш код здесь
for i in range(len(Xtrain.columns)):
    print(Xtrain.columns[i], model.feature_importances_[i])

year 0.19482654045858894
km_driven 0.014030951737050504
mileage 0.008783190827987014
engine 0.01927229762878075
max_power 0.7583703869174462
seats 0.004716632430146449
